In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [12]:
# Load libraries
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(r'src\rawDataset.csv')
df.head()

df.drop(columns=['RowNumber','Surname','CustomerId'], inplace= True)

#Codification
encoder = OneHotEncoder(sparse_output=False, drop='first', dtype=int)

catcols = ['Gender','Geography', 'Card Type']
encData = encoder.fit_transform(df[catcols])
encDF = pd.DataFrame(encData, columns=encoder.get_feature_names_out(catcols))
df = pd.concat([df.drop(columns= catcols),encDF],axis=1)

df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,Gender_Male,Geography_Germany,Geography_Spain,Card Type_GOLD,Card Type_PLATINUM,Card Type_SILVER
0,619,42,2,0.00,1,1,1,101348.88,1,1,2,464,0,0,0,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,3,456,0,0,1,0,0,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,3,377,0,0,0,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,5,350,0,0,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,5,425,0,0,1,1,0,0


# Feature Selection

### Filter Method

A subset of features is selected based on their relationship to the target variable. The selection is not dependent of any machine learning algorithm. On the contrary, filter methods measure the “relevance” of the features with the output via statistical tests.

### Wrapper Method

1- *Forward Selection*

In [13]:
decoded_categories = encoder.inverse_transform(encData)
forward_selection_df = pd.concat([df.drop(columns=encDF.columns), pd.DataFrame(decoded_categories, columns=catcols)], axis=1)

lblencoder = LabelEncoder()

for col in catcols:
    forward_selection_df[col] = lblencoder.fit_transform(forward_selection_df[col])

forward_selection_df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,Gender,Geography,Card Type
0,619,42,2,0.00,1,1,1,101348.88,1,1,2,464,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,3,456,0,2,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,3,377,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,5,350,0,0,1
4,850,43,2,125510.82,1,1,1,79084.10,0,0,5,425,0,2,1


For the forward selection we had to encode the categorical colums with a label encoder, so it doesnt choose some of the atributes of a feature instead of the whole feature, as the one hot encoder divides them in more features.

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import plotly.graph_objects as go
import numpy as np

X = forward_selection_df.drop(columns=['Exited'])
y = forward_selection_df['Exited']

selected_features = []
feature_names = list(X.columns)
scores_list = []

model = RandomForestClassifier(random_state=42)

accuracy_threshold = 0.001
max_no_improvement_iterations = 5
no_improvement_count = 0
best_score = -1

while len(selected_features) < 13:
    best_score = -1
    best_feature = None

    for feature_idx in range(X.shape[1]):
        if feature_idx in selected_features:
            continue

        candidate_features = selected_features + [feature_idx]
        candidate_feature_names = [feature_names[i] for i in candidate_features]

        # cross-validation
        scores = cross_val_score(model, X[candidate_feature_names], y, cv=5, scoring='accuracy')
        mean_score = np.mean(scores)

        # best-performing feature
        if mean_score > best_score:
            best_score = mean_score
            best_feature = feature_idx

    if best_feature is not None:
        selected_features.append(best_feature)
        scores_list.append(best_score)

        print(f"Selected Feature {len(selected_features)}: {feature_names[best_feature]}, Mean Accuracy: {best_score:.4f}")

        if abs(best_score - np.mean(cross_val_score(model, X[[feature_names[i] for i in selected_features]], y, cv=5))) < accuracy_threshold:
            no_improvement_count += 1
        else:
            no_improvement_count = 0
        
        # Stop if there's no improvement for a predefined number of iterations
        if no_improvement_count >= max_no_improvement_iterations:
            print("Stopping early due to lack of significant improvement.")
            break
        

Selected Feature 1: Complain, Mean Accuracy: 0.9986
Selected Feature 2: CreditScore, Mean Accuracy: 0.9986
Selected Feature 3: EstimatedSalary, Mean Accuracy: 0.9986
Selected Feature 4: Age, Mean Accuracy: 0.9986
Selected Feature 5: Tenure, Mean Accuracy: 0.9986
Stopping early due to lack of significant improvement.


In [36]:
import pandas as pd

selection_df = pd.DataFrame({
    "Selection Order": range(1, len(selected_features) + 1),
    "Feature Name": [feature_names[i] for i in selected_features]
})

selection_df.style.set_caption("Feature Selection Order")

,Selection Order,Feature Name
0,1,Complain
1,2,CreditScore
2,3,EstimatedSalary
3,4,Age
4,5,Tenure


In [ ]:
import plotly.graph_objects as go

table = go.Figure(data=[go.Table(
    header=dict(values=["<b>Selection Order</b>", "<b>Feature Name</b>"],
                fill_color='lightgrey',
                align='center'),
    cells=dict(values=[list(range(1, len(selected_features) + 1)),
                       [feature_names[i] for i in selected_features]],
               fill_color='white',
               align='center')
)])

table.update_layout(title="Feature Selection Order")
table.show()

2- *Backward Elimination*

In [ ]:
all_features = list(range(X.shape[1]))
min_features_to_retain = 5

removed_features = []
accuracy_scores = []

while len(all_features) > min_features_to_retain:
    worst_score = 1.0  
    worst_feature = None

    for feature_idx in all_features:

        candidate_features = [f for f in all_features if f != feature_idx]
        candidate_feature_names = [feature_names[i] for i in candidate_features]

        # cross-validation
        scores = cross_val_score(model, X[candidate_feature_names], y, cv=5, scoring='accuracy')
        mean_score = np.mean(scores)

        # worst-performing feature
        if mean_score < worst_score:
            worst_score = mean_score
            worst_feature = feature_idx

    if worst_feature is not None:
        all_features.remove(worst_feature)
        removed_features.append(feature_names[worst_feature])
        accuracy_scores.append(worst_score)
        print(f"Removed Feature : {feature_names[worst_feature]}, Mean Accuracy: {worst_score:.4f}")

print("Remaining feature indices:", {feature_names[i] for i in all_features})

Removed Feature : Complain, Mean Accuracy: 0.8614
Removed Feature : Age, Mean Accuracy: 0.8220
Removed Feature : NumOfProducts, Mean Accuracy: 0.7937
Removed Feature : Balance, Mean Accuracy: 0.7893
Removed Feature : EstimatedSalary, Mean Accuracy: 0.7818
Removed Feature : CreditScore, Mean Accuracy: 0.7584
Removed Feature : Tenure, Mean Accuracy: 0.7281
Removed Feature : IsActiveMember, Mean Accuracy: 0.7066
Removed Feature : Geography, Mean Accuracy: 0.6922
Remaining feature indices: {'Point Earned', 'HasCrCard', 'Gender', 'Satisfaction Score', 'Card Type'}


In [ ]:
table = go.Figure(data=[go.Table(
    header=dict(values=["<b>Feature Name</b>"],
                fill_color='lightgrey',
                align='center'),
    cells=dict(values=[[feature_names[i] for i in all_features]],
               fill_color='white',
               align='center')
)])


table.update_layout(title="Feature Selection Order")
table.show()

In [19]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, len(removed_features) + 1)),
    y=accuracy_scores,
    mode='lines+markers',
    text=removed_features,
    hovertemplate='<b>Step %{x}</b><br>Removed Feature: %{text}<br>Mean Accuracy: %{y:.4f}<extra></extra>',
    name='Backward Elimination'
))

fig.update_layout(
    title="Backward Elimination Process",
    xaxis_title="Number of Features Removed",
    yaxis_title="Mean Accuracy",
    hovermode="closest",
    template="plotly_white"
)

fig.show()

3- *Selection Model*

In [8]:
from sklearn.feature_selection import SelectFromModel

sfm_selector = SelectFromModel(estimator=model)
sfm_selector.fit(X, y)
X.columns[sfm_selector.get_support()]

Index(['Age', 'Complain'], dtype='object')

3.2- *Feature Selection Sequential Feature Selection (SFS)*

In [18]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import VarianceThreshold

threshold = 0.01
vt = VarianceThreshold(threshold=threshold)
X_reduced = vt.fit_transform(X)

sfs_selector = SequentialFeatureSelector(estimator=model, n_features_to_select = 3, cv =5, direction ='backward', n_jobs=-1)
sfs_selector.fit(X_reduced, y)
X.columns[sfs_selector.get_support()]

Index(['EstimatedSalary', 'Complain', 'Point Earned'], dtype='object')

4- *Recursive Feature Elimination (RFE)*

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np


num_features_to_retain = 5
rfe = RFE(estimator=model, n_features_to_select=num_features_to_retain)

# Fit the RFE
rfe.fit(X, y)


selected_features = np.where(rfe.support_)[0]

print("Selected feature indices:", [feature_names[i] for i in selected_features])

# cross-validation
scores = cross_val_score(model, X[[feature_names[i] for i in selected_features]], y, cv=5, scoring='accuracy')
mean_accuracy = np.mean(scores)
print(f"Mean Accuracy with Selected Features: {mean_accuracy:.4f}")

Selected feature indices: ['Age', 'Balance', 'NumOfProducts', 'IsActiveMember', 'Complain']
Mean Accuracy with Selected Features: 0.9985


In [42]:
# selected_features = ['Age', 'Balance', 'NumOfProducts', 'IsActiveMember', 'Complain']
# mean_accuracy = 0.9985

fig = go.Figure(data=[go.Table(
    header=dict(
        values=["<b>Selected Features</b>", "<b>Mean Accuracy</b>"],
        fill_color="lightblue",
        align="center",
        font=dict(size=14, color="black")
    ),
    cells=dict(
        values=[[feature_names[i] for i in selected_features], [f"{mean_accuracy:.4f}"]],
        fill_color="white",
        align="center",
        font=dict(size=12)
    )
)])

# Estilizar y mostrar la tabla
fig.update_layout(
    title="Selected Features and Model Accuracy",
    title_x=0.5,
    template="plotly_white",
    height=400
)

fig.show()

4- *Exhaustive Search*

In [48]:
from itertools import combinations

max_features = 5

# Initialize variables to keep track of the best feature subset and its accuracy
best_subset = None
best_accuracy = 0.0

# Generate all possible combinations of feature indices
all_feature_combinations = list(combinations(range(X.shape[1]), max_features))

for feature_subset in all_feature_combinations:
    feature_subset = list(feature_subset)
    
    # Evaluate the model's performance using cross-validation
    X_subset = X.iloc[:, feature_subset]
    scores = cross_val_score(model, X_subset, y, cv=5, scoring='accuracy')
    mean_accuracy = np.mean(scores)

    # Check if this feature subset is better than the best one found so far
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_subset = feature_subset

print("Best Feature Subset:", best_subset)
print("Best Accuracy:", best_accuracy)

Best Feature Subset: [0, 1, 2, 3, 8]
Best Accuracy: 0.9986


### Embedded Method